In [16]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from io import BytesIO

In [17]:
def getcsv(date_possible):
    url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    payload = {
        "locale": "ko_KR",
        "trdDd": date_possible,
        "share": "1",
        "money": "1",
        "csvxls_isNo": "false",
        "name": "fileDown",
        "url": "dbms/MDC/STAT/standard/MDCSTAT04301"
    }
    headers = {
        'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'
    }
    code = rq.post(url,payload,headers = headers).text

    down_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
    down_payload = {
        "code" : code
    }
    headers = {
        'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'
    }
    etf_price = rq.post(down_url , down_payload , headers = headers)

    csv = pd.read_csv(BytesIO(etf_price.content) , encoding = 'EUC_KR')
    csv['날짜'] = date_possible
    return csv[['종목코드', '시가총액','날짜']]




# 담을 데이터 프레임 생성

In [18]:
columns = ['종목코드', '시가총액', '날짜']
etf_price = pd.DataFrame(columns = columns)
print(etf_price)

Empty DataFrame
Columns: [종목코드, 시가총액, 날짜]
Index: []


In [19]:
from datetime import date
from datetime import datetime, timedelta
import holidays
from itertools import chain
from tqdm import tqdm
import time

kr_holidays = holidays.KR()

date_str = "20240331"

date_obj = datetime.strptime(date_str,"%Y%m%d").date()

for year in tqdm(range(6)):

    while (date_obj in kr_holidays or date_obj.weekday() >= 5):
        date_obj -= timedelta(days=1)
    date_possible = date_obj.strftime("%Y%m%d")

    df = getcsv(date_possible)
    etf_price = pd.concat([df, etf_price])
        
    date_obj = date(2023-year,3,31)
    time.sleep(1)
    
etf_price = etf_price.reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.61s/it]


In [20]:
etf_price = etf_price.rename(columns = {'종목코드' : '단축코드'})

In [21]:
etf_price

,단축코드,시가총액,날짜
0,152100,827604000000,20190329
1,295820,6338500000,20190329
2,295890,8990000000,20190329
3,295860,5599750000,20190329
4,295840,6603750000,20190329
...,...,...,...
3421,429870,4875000000,20240329
3422,476450,380127550000,20240329
3423,460270,8930250000,20240329
3424,459790,9360000000,20240329


In [22]:
import pymysql
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://user:password@ip:3306/db')

etf_ticker = pd.read_sql("""
select * from etf_info
""",con = engine)

etf_ticker

,표준코드,단축코드,종목명,상장일
0,KR7069500007,69500,삼성 KODEX200 증권상장지수투자신탁[주식],2002-10-14
1,KR7069660009,69660,키움 KOSEF 200 상장지수증권투자신탁[주식],2002-10-14
2,KR7091160002,91160,삼성 KODEX 반도체 증권상장지수투자신탁[주식],2006-06-27
3,KR7091170001,91170,삼성 KODEX 은행 증권상장지수투자신탁[주식],2006-06-27
4,KR7091180000,91180,삼성 KODEX 자동차 증권상장지수투자신탁[주식],2006-06-27
...,...,...,...,...
851,KR7480260009,480260,미래에셋 TIGER 27-04회사채(A+이상)액티브증권상장지수투자신탁(채권),2024-04-23
852,KR7480310002,480310,미래에셋 TIGER 글로벌온디바이스AI증권상장지수투자신탁(주식),2024-04-16
853,KR7480460005,480460,우리 WOORI 한국부동산TOP3플러스부동산상장지수투자신탁[재간접형],2024-04-30
854,KR7481050003,481050,삼성 KODEX 1년은행CD플러스액티브증권상장지수투자신탁[채권혼합-파생형](합성),2024-04-23


In [23]:
etf_price['단축코드'] = etf_price['단축코드'].astype(str)

In [24]:
def get_standard_code(code):
    filter = etf_ticker['단축코드'] == code
    standard_code = etf_ticker.loc[filter,'표준코드'].values
    if len(standard_code) > 0:
        return standard_code[0]
    else:
        return None  # 또는 다른 값을 반환하여 해당 코드에 해당하는 표준코드가 없음을 나타낼 수 있습니다.


etf_price['표준코드'] = etf_price['단축코드'].apply(get_standard_code)

In [25]:
etf_price

,단축코드,시가총액,날짜,표준코드
0,152100,827604000000,20190329,KR7152100004
1,295820,6338500000,20190329,KR7295820005
2,295890,8990000000,20190329,None
3,295860,5599750000,20190329,None
4,295840,6603750000,20190329,None
...,...,...,...,...
3421,429870,4875000000,20240329,KR7429870009
3422,476450,380127550000,20240329,KR7476450002
3423,460270,8930250000,20240329,KR7460270002
3424,459790,9360000000,20240329,KR7459790002


In [26]:
filter = etf_price['표준코드'].isna()
etf_price[filter]

,단축코드,시가총액,날짜,표준코드
2,295890,8990000000,20190329,None
3,295860,5599750000,20190329,None
4,295840,6603750000,20190329,None
7,295880,6202000000,20190329,None
8,278420,20666250000,20190329,None
...,...,...,...,...
2560,137930,7470000000,20230331,None
2563,292340,4323000000,20230331,None
2814,368200,1293000000,20240329,None
2825,307010,6504000000,20240329,None


#### 표준코드 null인거 없애기
#### 근거는 표준코드가 null인거는 이미 나의 데이터 셋에 없는 데이터 임

In [27]:
etf_price = etf_price[~filter]

In [28]:
etf_price

,단축코드,시가총액,날짜,표준코드
0,152100,827604000000,20190329,KR7152100004
1,295820,6338500000,20190329,KR7295820005
5,253150,7980000000,20190329,KR7253150007
6,253160,3678750000,20190329,KR7253160006
9,292750,37990000000,20190329,KR7292750007
...,...,...,...,...
3421,429870,4875000000,20240329,KR7429870009
3422,476450,380127550000,20240329,KR7476450002
3423,460270,8930250000,20240329,KR7460270002
3424,459790,9360000000,20240329,KR7459790002


In [29]:
etf_price.to_csv('etf가격정보.csv',index = False)

In [30]:
pd.read_csv('etf가격정보.csv')

,단축코드,시가총액,날짜,표준코드
0,152100,827604000000,20190329,KR7152100004
1,295820,6338500000,20190329,KR7295820005
2,253150,7980000000,20190329,KR7253150007
3,253160,3678750000,20190329,KR7253160006
4,292750,37990000000,20190329,KR7292750007
...,...,...,...,...
3209,429870,4875000000,20240329,KR7429870009
3210,476450,380127550000,20240329,KR7476450002
3211,460270,8930250000,20240329,KR7460270002
3212,459790,9360000000,20240329,KR7459790002
